In [50]:
import ccxt
import pandas as pd
import time
import numpy as np
import plotly.express as px
import chart_studio.plotly
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import chart_studio.plotly as py
from datetime import datetime

In [51]:
ftx = ccxt.ftx({
    'apiKey': 'your_API',
    'secret': 'your_SECRET',
    'rateLimit': 200,
    'enableRateLimit': True
})

In [52]:
#Loading all ftx markets
markets = ftx.load_markets()

In [54]:
#parsing markets to keep perp usd pairs and remove bull and bear pairs
symbols = []
for item in markets.items(): 
    #print(type(item))
    for key in item:
        if type(key) == str:
            if ((key[-3:] == "USD") or (key[-4:] == "PERP")) and ("BEAR" not in key) and ("BULL" not in key):
                symbols.append(key)


In [55]:
#symbols Restrived
len(symbols)

468

In [ ]:
#Set the amount of candles to download from date of "from_datetime" untill now.
#To specify a different intervall uncomment the commented lines and comment now = ftx.milliseconds() and to_datetime = datetime.now()

msec = 1000
minute = 60 * msec
hold = 30
from_datetime = '2021-12-01 00:00:00'
from_timestamp = ftx.parse8601(from_datetime)

#to_datetime = '2021-12-02 00:00:00'
#to_timestamp = ftx.parse8601(to_datetime)

to_datetime = datetime.now()
to_timestamp = ftx.milliseconds()
df = pd.DataFrame()

#to specify a different list uncomment the following line that will override what is downloaded from ftx
#symbols = ['BTC/USD','ETH/USD','SOL/USD','XRP/USD','BNB/USD','FTT/USD','MATIC/USD','YFI/USD','LTC/USD','AVAX/USD','DOGE/USD','LINK/USD','SHIB/USD','SPELL/USD','ATLAS/USD','RAY/USD','MANA/USD','SUSHI/USD','SAND/USD','AXS/USD','RUNE/USD','CRV/USD','FTM/USD','SRM/USD','DYDX/USD','UNI/USD','OMG/USD','AAVE/USD','BCH/USD','POLIS/USD','CRO/USD','TRX/USD','BOBA/USD','BIT/USD','PERP/USD','GRT/USD','GODS/USD','SXP/USD','1INCH/USD','IMX/USD','ENJ/USD','MAPS/USD','RSR/USD','STEP/USD','COMP/USD','GALA/USD','BAT/USD','GENE/USD','HNT/USD','KIN/USD','HT/USD','BICO/USD','ENS/USD','TRU/USD','AURY/USD','ALICE/USD','LRC/USD','FIDA/USD','SLP/USD','CHZ/USD','OXY/USD','OKB/USD','TOMO/USD','CEL/USD','BNT/USD','MKR/USD','RNDR/USD','AMPL/USD','SNX/USD','DFL/USD','TONCOIN/USD','MNGO/USD','PUNDIX/USD','REEF/USD','MOB/USD','BADGER/USD','ROOK/USD','AUDIO/USD','CQT/USD','ZRX/USD','YFII/USD','REN/USD','ALCX/USD','STORJ/USD','CHR/USD','STARS/USD','CREAM/USD','KNC/USD','SLND/USD','TULIP/USD','ALPHA/USD','SKL/USD','SECO/USD','COPE/USD','VGX/USD','HUM/USD','LEO/USD','STMX/USD','QI/USD','WAVES/USD','INTER/USD','BAND/USD','BAO/USD','DODO/USD','BAL/USD','DAWN/USD','MTA/USD','C98/USD','MER/USD','CLV/USD','LUA/USD']

def get_candles(_symbol,_start_timestamp,_end_timestamp):
    _data = []
    while _start_timestamp < _end_timestamp:
    
        try:

            print(ftx.milliseconds(), 'Fetching candles starting from', ftx.iso8601(_start_timestamp))
            ohlcvs = ftx.fetch_ohlcv(_symbol, '1m', _start_timestamp)
            print(ftx.milliseconds(), 'Fetched', len(ohlcvs), 'candles')
            first = ohlcvs[0][0]
            last = ohlcvs[-1][0]
            print('First candle epoch', first, ftx.iso8601(first))
            print('Last candle epoch', last, ftx.iso8601(last))
            _start_timestamp += len(ohlcvs) * minute
            _data += ohlcvs
            

        except (ccxt.ExchangeError, ccxt.AuthenticationError, ccxt.ExchangeNotAvailable, ccxt.RequestTimeout) as error:

            print('Got an error', type(error).__name__, error.args, ', retrying in', hold, 'seconds...')
            time.sleep(hold)
    return _data

for symbol in symbols:
    print('Fetching Candles for ',symbol)
    try:
        df = pd.DataFrame(get_candles(symbol,from_timestamp,to_timestamp), columns=['time', 'open', 'high', 'low', 'close', 'volume'])
        df['Volatility %']=(df['high']-df['low'])/df['close']*100   
        df['Normalized Close']=(df['close']-df['close'].mean())/df['close'].std()
        coefficients, residuals, _, _, _ = np.polyfit(range(len(df.index)),df['Normalized Close'],1,full=True)
        mse = residuals[0]/(len(df.index))
        nrmse = np.sqrt(mse)/(df['Normalized Close'].max() - df['Normalized Close'].min())
        print('Slope ' + str(coefficients[0]))
        print('NRMSE: ' + str(nrmse))
        one_pct_candles = df[df['Volatility %']>1]['Volatility %'].count()
    
        if symbol == symbols[0]:
            data={'Symbol':[symbol],'Volatility':[df.mean()['Volatility %']],'Variance':[df.var()['close']],'Volume':[df.mean()['volume']],'Linear Trend %':[coefficients[0]*100],'1% Candles':[one_pct_candles]}
            metrics = pd.DataFrame(data)
            print(data)
        else:
            data={'Symbol':symbol,'Volatility':df.mean()['Volatility %'],'Variance':df.var()['close'],'Volume':df.mean()['volume'],'Linear Trend %':coefficients[0]*100,'1% Candles':one_pct_candles}
            metrics = metrics.append(data,ignore_index=True)
            print(data)
    except:
        print("An exception occurred on symbol ",symbol)
    

    
    


Fetching Candles for  1INCH-PERP
1640164671796 Fetching candles starting from 2021-12-01T00:00:00.000Z
1640164672522 Fetched 1501 candles
First candle epoch 1638316800000 2021-12-01T00:00:00.000Z
Last candle epoch 1638406800000 2021-12-02T01:00:00.000Z
1640164672523 Fetching candles starting from 2021-12-02T01:01:00.000Z
1640164673163 Fetched 1501 candles
First candle epoch 1638406860000 2021-12-02T01:01:00.000Z
Last candle epoch 1638496860000 2021-12-03T02:01:00.000Z
1640164673163 Fetching candles starting from 2021-12-03T02:02:00.000Z
1640164673836 Fetched 1501 candles
First candle epoch 1638496920000 2021-12-03T02:02:00.000Z
Last candle epoch 1638586920000 2021-12-04T03:02:00.000Z
1640164673837 Fetching candles starting from 2021-12-04T03:03:00.000Z
1640164674502 Fetched 1490 candles
First candle epoch 1638586980000 2021-12-04T03:03:00.000Z
Last candle epoch 1638677040000 2021-12-05T04:04:00.000Z
1640164674503 Fetching candles starting from 2021-12-05T03:53:00.000Z
1640164675150 Fet

In [42]:
metrics

,Symbol,Volatility,Variance,Volume,Linear Trend %,1% Candles
0,1INCH-PERP,0.139553,0.001871,6098.702336,0.075398,1
1,1INCH/USD,0.131337,0.001876,1401.459365,0.075598,1
2,AAPL/USD,0.057355,2.499312,60.432767,0.054914,0
3,AAVE-PERP,0.145052,43.629787,13420.231219,0.075090,2
4,AAVE/USD,0.115788,43.652412,3707.469942,0.075180,2
...,...,...,...,...,...,...
550,XTZBULL/USD,0.198715,0.000001,50.258415,0.074441,146
551,XTZHALF/USD,0.081010,10457.389485,0.000000,0.076493,2
552,XTZHEDGE/USD,0.134441,2.947828,0.023920,-0.071148,7
553,ZECBEAR/USD,0.309405,0.000017,3.954996,-0.066701,359


In [46]:
Title_1 = "Coin Volatility vs Variance Volume and Trend from "+str(from_datetime)+" to "+str(to_datetime)
fig = px.scatter(metrics, x="Variance", y="Volatility",
	         size="Volume", color="Linear Trend %", hover_name="Symbol", log_x=True, size_max=80,template="simple_white", title=Title_1)

fig.update_xaxes(showgrid=True, showline=True, linewidth=2, linecolor='black', mirror=True)
fig.update_yaxes(showgrid=True, showline=True, linewidth=2, linecolor='black', mirror=True)
fig.show()


'https://plotly.com/~riodda/15/'

In [47]:
Title_2 = "Coin n° >1% candle vs Variance Volume and Trend from "+str(from_datetime)+" to "+str(to_datetime)
fig = px.scatter(metrics, x="Variance", y="1% Candles",
	         size="Volume", color="Linear Trend %", hover_name="Symbol", log_x=True, size_max=60,template="simple_white", title=Title_2)

fig.update_xaxes(showgrid=True, showline=True, linewidth=2, linecolor='black', mirror=True)
fig.update_yaxes(showgrid=True, showline=True, linewidth=2, linecolor='black', mirror=True)
fig.show()


'https://plotly.com/~riodda/17/'